## 作業
礙於不是所有同學都有 GPU ，這邊的範例使用的是簡化版本的 ResNet，確保所有同學都能夠順利訓練!


最後一天的作業請閱讀這篇非常詳盡的[文章](https://blog.gtwang.org/programming/keras-resnet-50-pre-trained-model-build-dogs-cats-image-classification-system/)，基本上已經涵蓋了所有訓練　CNN 常用的技巧，請使用所有學過的訓練技巧，盡可能地提高 Cifar-10 的 test data 準確率，截圖你最佳的結果並上傳來完成最後一次的作業吧!

另外這些技巧在 Kaggle 上也會被許多人使用，更有人會開發一些新的技巧，例如使把預訓練在 ImageNet 上的模型當成 feature extractor 後，再拿擷取出的特徵重新訓練新的模型，這些技巧再進階的課程我們會在提到，有興趣的同學也可以[參考](https://www.kaggle.com/insaff/img-feature-extraction-with-pretrained-resnet)

In [0]:
from keras.datasets import cifar10

from keras.models import Model
from keras.optimizers import Adam
from keras.utils import to_categorical
from keras.layers import Dense,Dropout

In [28]:
from google.colab import files
src = list(files.upload().values())[0]
open('resnet_builder.py','wb').write(src)

from resnet_builder import resnet # 這是從 resnet_builder.py 中直接 import 撰寫好的 resnet 函數


Saving resnet_builder.py to resnet_builder (1).py


In [29]:
# 讀取資料集並作前處理
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

x_train = x_train / 255.
x_test = x_test / 255.
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [30]:
# 建立 ResNet 模型
model = resnet(input_shape=(32,32,3),num_classes=10) 
model.summary()

model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_7 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_187 (Conv2D)             (None, 32, 32, 16)   448         input_7[0][0]                    
__________________________________________________________________________________________________
batch_normalization_169 (BatchN (None, 32, 32, 16)   64          conv2d_187[0][0]                 
__________________________________________________________________________________________________
activation_169 (Activation)     (None, 32, 32, 16)   0           batch_normalization_169[0][0]    
__________________________________________________________________________________________________
conv2d_188

In [0]:
from keras.preprocessing.image import ImageDataGenerator
data_generator = ImageDataGenerator( rotation_range=20, width_shift_range=0.2,
                                     height_shift_range=0.2,
                                     horizontal_flip=True)

from keras.callbacks import EarlyStopping
earlystop = EarlyStopping(monitor="val_loss", patience=10,verbose=1 )

from keras.callbacks import ReduceLROnPlateau
reduce_lr = ReduceLROnPlateau(factor=0.3,min_lr=1e-12, monitor='val_loss', patience=5, verbose=1)


In [35]:
batch_size = 128
epochs = 100 # 訓練整個資料集共 50個循環


model.fit_generator(data_generator.flow(x_train, y_train, batch_size=batch_size),
                    steps_per_epoch=len(x_train) / batch_size, epochs=epochs,
                    callbacks=[earlystop,reduce_lr],
                    )

Epoch 1/100
391/390 [==============================] - 35s 90ms/step - loss: 0.5169 - acc: 0.8973
Epoch 2/100
  2/390 [..............................] - ETA: 33s - loss: 0.5442 - acc: 0.8594

/usr/local/lib/python3.6/dist-packages/keras/callbacks.py:569: RuntimeWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: loss,acc
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning
/usr/local/lib/python3.6/dist-packages/keras/callbacks.py:1109: RuntimeWarning: Reduce LR on plateau conditioned on metric `val_loss` which is not available. Available metrics are: loss,acc,lr
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


391/390 [==============================] - 34s 88ms/step - loss: 0.4987 - acc: 0.9011
Epoch 3/100
391/390 [==============================] - 34s 87ms/step - loss: 0.5021 - acc: 0.8984
Epoch 4/100
391/390 [==============================] - 34s 88ms/step - loss: 0.4965 - acc: 0.8998
Epoch 5/100
391/390 [==============================] - 35s 88ms/step - loss: 0.5004 - acc: 0.8975
Epoch 6/100
391/390 [==============================] - 34s 88ms/step - loss: 0.4922 - acc: 0.9012
Epoch 7/100
391/390 [==============================] - 34s 87ms/step - loss: 0.4899 - acc: 0.8979
Epoch 8/100
391/390 [==============================] - 34s 88ms/step - loss: 0.4915 - acc: 0.8985
Epoch 9/100
391/390 [==============================] - 34s 87ms/step - loss: 0.4905 - acc: 0.8990
Epoch 10/100
391/390 [==============================] - 34s 88ms/step - loss: 0.4864 - acc: 0.8973
Epoch 11/100
391/390 [==============================] - 34s 87ms/step - loss: 0.4867 - acc: 0.8997
Epoch 12/100
391/390 [========

In [36]:
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.6530581028938294
Test accuracy: 0.8661
